In [68]:
import os
import pandas as pd
#import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import re
import numpy as np
#os.getcwd()

In [69]:
comentarios = pd.read_csv('C:/Users/Kikes/Documents/GitHub/NLP/df_comms.csv', encoding='utf-8', index_col=0)

In [70]:
df = comentarios.copy().head(10)

In [71]:
df['solo_emoji'] = np.where(df.comm_msg.isnull()&df.emojis.notnull(),1,0)
#comentarios[comentarios.solo_emoji == 1]

In [72]:
#dropeo filas con comentarios nulos
df.dropna( subset = ['comm_msg'], inplace = True )

In [73]:
#Desde acá: https://nlp.stanford.edu/software/CRF-NER.shtml#Download
#Se baja: https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
#Se descomprime el .zip y a la carpeta que está adentro se le cambia el nombre de: stanford-ner-2017-06-09 a: stanford-ner 

#import os
#java_path = "C:/Program Files/Java/jre1.8.0_131/bin/java.exe"
java_path = 'C:/Program Files (x86)/Java/jre1.8.0_131/bin/java.exe'

os.environ['JAVAHOME'] = java_path

#os.getcwd()
#Hay que cambiar el directorio y que apunte donde está la carpeta que descomprimimos
#os.chdir("C:/Users/axmarr/Documents/Varios/maestria/NLP/")
os.chdir('C:/Users/Kikes/Documents/maestria/NLP/')

# -*- coding: utf-8 -*-

#from nltk.tag import StanfordNERTagger
#from nltk.tokenize import word_tokenize

st = StanfordNERTagger('stanford-ner/classifiers/spanish.ancora.distsim.s512.crf.ser.gz',
					   'stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

In [74]:
def extraeNombres(comm):
    comm_str = str(comm)#comm.to_json()
    #comm_token = word_tokenize(comm_str)
    comm_token = word_tokenize(comm_str)
    #comm_token = word_tokenize(comm)
    comm_class = st.tag(comm_token)
    comm_list_per = [pers for pers in comm_class if pers[1] =='PERS']
    comm_per = re.sub(r'[\[\]\(\)\'\,]', "", str([i[0] for i in comm_list_per]))
    return comm_per

In [75]:
#Ver como optimizar este punto, el cuello de botella esta en st.tag()
#https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

pd.options.mode.chained_assignment = None
df['nombres'] = df['comm_msg'].apply(lambda x: extraeNombres(x))
#comentarios['nombres'] = comentarios['comm_msg'].apply(lambda x: extraeNombres(x))
pd.options.mode.chained_assignment = 'warn'

In [76]:
n_comm = len(df)

for c in range(n_comm):
    ext_name = str(df.nombres[c])
    ext_comm = str(df.comm_msg[c])
    comm_no_name = ext_comm.replace(ext_name,'')
    df.comm_msg[c] = comm_no_name


C:\Users\Kikes\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
df['solo_nombre'] = np.where( df.comm_msg.isnull()&df.nombres.notnull()&df.emojis.notnull(), 1, 0 )

In [78]:
df.dropna( subset = ['comm_msg'], inplace = True )

In [79]:
df['largo_comm'] = df['comm_msg'].apply(lambda x: len(x.replace(" ","")))

In [80]:
df['comm_msg'] = df['comm_msg'].apply(lambda x: re.sub(r'[!#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~¡]', " ", x))

In [84]:
#https://stackoverflow.com/questions/19130512/stopword-removal-with-nltk
from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))
sw2 = set(line.strip() for line in open('stop_words_acentos.txt'))
stop2 = stop|sw2

In [159]:
#comm  = df['comm_msg'][1]

def quitaSW(comm):

    split_comm = comm.lower().split()
    comm_no_stop = [i for i in split_comm if i not in stop2]

    return comm_no_stop

In [160]:
df['comm_msg'] = df['comm_msg'].apply( lambda x: quitaSW( x ) )

AttributeError: 'list' object has no attribute 'lower'

In [151]:
df

,post_id,comm_id,comm_msg,comm_date,emojis,emoji_count,solo_emoji,nombres,solo_nombre,largo_comm
0,262755230730537_546328889039835,546328889039835_546416972364360,"[mmm, rico]",2017-11-22 19:03:36,NaN,0,0,,0,10
1,262755230730537_546317689040955,546317689040955_546341119038612,"[copa, postres, usan, frascos, indignación, da...",2017-11-22 15:14:52,😵😂😂😂,4,0,,0,63
2,262755230730537_546317689040955,546317689040955_546422655697125,"[mira, navidad, vos, veras, reemplazar, licor,...",2017-11-22 19:26:31,😱,1,0,Melina Yasmin,0,52
3,262755230730537_546317689040955,546317689040955_546339792372078,"[q, rico, noo, fresquito, verano, asik, pronti...",2017-11-22 15:09:30,😄,1,0,Majo Maciel,0,59
4,262755230730537_546317689040955,546317689040955_546346215704769,"[tendencia, veo, moso, trayendo, semejante, po...",2017-11-22 15:39:09,NaN,0,0,,0,91
5,262755230730537_546317689040955,546317689040955_546344875704903,"[nooooo, jajajajaja]",2017-11-22 15:33:04,NaN,0,0,Siiiiiiii,0,29
6,262755230730537_546317689040955,546317689040955_546436992362358,"[engordo, feliz, jaja]",2017-11-22 20:08:28,NaN,0,0,Abalos Yanina,0,22
7,262755230730537_546317689040955,546317689040955_546344252371632,"[postre, super, frascos, queda, horrible]",2017-11-22 15:30:07,NaN,0,0,,0,49
8,262755230730537_546317689040955,546317689040955_546338485705542,"[i, love, brownie]",2017-11-22 15:05:36,NaN,0,0,,0,12
9,262755230730537_546317689040955,546317689040955_546346609038063,"[ve, comodisimo, comer, eh, ajjajajaja, malisimo]",2017-11-22 15:41:07,NaN,0,0,,0,51


In [82]:
#os.chdir('C:/Users/Kikes/Documents/maestria/NLP/')
#df_com = pd.read_csv('comentarios.csv', encoding='utf-8', index_col = 0)
#df_com.head(10)